In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score, auc, precision_score, classification_report, confusion_matrix, accuracy_score, recall_score, f1_score
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import scipy.io as sp
from sklearn.decomposition import PCA
from sklearn.linear_model import Perceptron
from sklearn.calibration import CalibratedClassifierCV
import xlsxwriter
import pandas as pd
from IPython.display import display
from sklearn.model_selection import StratifiedKFold




#Traitement du fichier .mat
cancercell = sp.loadmat('breastw.mat')
X = cancercell['X']
y = cancercell['y']
#print(cancercell)
y=np.reshape(y,(683,))
#fin traitement mat

In [20]:
penalty = ['elasticnet']
Max_iter = []
for k in range (1,15):
    Max_iter.append(50*k)
l1_ratio =[0.1, 0.001, 0.0001, 0.00001, 0.000001] 
Alpha = []
for k in range (0,7):
    Alpha.append(10**(-k))

#data = np.array([['penalty','Max_iter','Eta','Alpha','F1 score','recall','precision','accuracy','AUC',]])

skf = StratifiedKFold(n_splits=10,shuffle=True) # équilibrage dataset
algo = 0

for j in l1_ratio :
    for p in Alpha : 
        test =False
        for train_index, test_index in skf.split(X,y):
            X_train, X_test = X[train_index,:], X[test_index,:]
            y_train, y_test = y[train_index], y[test_index]
            y_train=np.ravel(y_train)
            y_test=np.ravel(y_test)

            #algo Perceptron
            model_Percep = Perceptron(penalty='elasticnet', l1_ratio=j, alpha=p)
            model_Percep.fit(X_train,y_train)
            y_pred_percep = model_Percep.predict(X_test)
                
            #fin algo Perceptron
            
            #Scores 
            F1_Percep = round(f1_score(y_test,y_pred_percep),3)
            
            #Construction matrice hyperparametres et scores
            if test == False :
                matrice_Percep = np.array([F1_Percep])
                test = True
            else :
                matrice_Percep = np.append(matrice_Percep,np.array([F1_Percep]).reshape(1,1))
                
        if algo == 0  :

            df=pd.DataFrame(matrice_Percep,columns=['[{},{}]'.format(j,p)])

        else :

            df1=pd.DataFrame(matrice_Percep,columns=['[{},{}'.format(j,p)])
            df=pd.concat([df, df1], axis = 1)

        algo = algo + 1

display(df)



#Dataframe 
"""
df = pd.DataFrame(matrice_Percep, columns=['penalty','l1_ratio','Alpha','F1 score'])
df = df.astype({'F1 score': float})
filtered_values_1 = np.where((df['F1 score']>=0.95))
display(df.iloc[filtered_values_1].sort_values(by=['F1 score'], ascending=False))
"""

,"[0.1,1]","[0.1,0.1","[0.1,0.01","[0.1,0.001","[0.1,0.0001","[0.1,1e-05","[0.1,1e-06","[0.001,1","[0.001,0.1","[0.001,0.01",...,"[1e-05,0.0001","[1e-05,1e-05","[1e-05,1e-06","[1e-06,1","[1e-06,0.1","[1e-06,0.01","[1e-06,0.001","[1e-06,0.0001","[1e-06,1e-05","[1e-06,1e-06"
0,0.000,0.960,0.960,0.960,0.884,0.906,0.889,0.000,0.814,0.769,...,0.939,0.960,0.906,0.000,0.222,0.906,0.980,0.923,0.222,0.889
1,0.000,0.588,0.902,0.800,0.898,0.842,0.906,0.000,0.917,0.829,...,0.923,0.873,0.957,0.000,0.960,0.894,0.958,0.960,0.960,1.000
2,0.000,0.980,0.980,0.960,0.571,0.958,0.960,0.516,0.941,0.958,...,0.941,0.960,0.884,0.000,0.980,0.980,0.958,0.941,0.906,0.080
3,0.000,0.933,0.939,0.979,0.979,0.920,0.920,0.000,0.757,0.958,...,0.920,0.979,0.930,0.000,0.909,0.868,0.885,0.936,1.000,0.933
4,0.000,0.939,0.980,0.828,0.979,0.941,0.884,0.000,0.979,0.939,...,0.958,0.898,1.000,0.522,0.980,0.941,0.958,0.936,0.980,0.980
5,0.000,0.884,0.941,0.913,0.829,0.923,0.960,0.000,0.545,0.750,...,0.979,0.960,0.889,0.000,0.000,0.941,0.980,1.000,0.960,0.889
6,0.000,0.222,0.913,0.979,0.960,0.958,0.979,0.000,0.980,0.936,...,0.960,0.923,0.980,0.000,0.667,0.939,0.939,0.958,0.960,0.920
7,0.522,0.941,0.960,0.957,0.737,0.980,0.939,0.000,0.941,0.864,...,0.941,1.000,0.980,0.000,0.980,0.958,0.980,0.941,0.960,0.957
8,0.000,0.889,0.957,0.923,0.941,0.939,0.923,0.000,0.884,0.957,...,0.960,0.939,0.906,0.000,0.080,0.829,0.957,1.000,0.980,0.941
9,0.522,0.762,0.941,0.957,0.980,1.000,0.718,0.000,0.958,0.958,...,0.980,0.957,0.958,0.000,0.750,0.980,0.833,0.960,0.920,0.920


"\ndf = pd.DataFrame(matrice_Percep, columns=['penalty','l1_ratio','Alpha','F1 score'])\ndf = df.astype({'F1 score': float})\nfiltered_values_1 = np.where((df['F1 score']>=0.95))\ndisplay(df.iloc[filtered_values_1].sort_values(by=['F1 score'], ascending=False))\n"